**177312 - Tales Lelo da Aparecida**

1\. Pre-processamento
  + Com a função **get_dummies** converta a primeira coluna de dados categóricos para numéricos.
  + Transforme a última coluna em um dado categórico, onde a classe de saída será $1$ se for maior que $13$, caso contrário $0$, depois remova-a.
  + Separe os primeiros $3133$ dados como treino e use os restantes como teste.
  

In [3]:
import pandas as pd
from sklearn.linear_model  import LogisticRegression
from sklearn.pipeline      import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer

# Carrega um DataFrame com os dados do arquivo
df = pd.io.parsers.read_csv('abalone.csv', ',', header=None )
#                                                                       .
# Converte a 1ª coluna para um dado numérico
df = pd.get_dummies(df, columns=[0])

# Cria um DataFrame com a versão categórica (1:x>13, 0:x<=13) da coluna 8
maiorQueOito = df.apply(lambda row: int(row[8]>13), 1)
# Remove-a
del df[8]

# Separando dados para treino e teste
treino, y_treino = df[:3133].as_matrix(), maiorQueOito[:3133].as_matrix()
teste, y_teste = df[3133:].as_matrix(), maiorQueOito[3133:].as_matrix()

# Renomeia as colunas para facilitar o acompanhamento
df.columns = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'F', 'I', 'M']

# Mostra as primeiras linhas de nossos dados, para um preview de como estão até então
print("Primeiras 5 linhas:\n", df.head())

Primeiras 5 linhas:
       V1     V2     V3      V4      V5      V6     V7  F  I  M
0  0.455  0.365  0.095  0.5140  0.2245  0.1010  0.150  0  0  1
1  0.350  0.265  0.090  0.2255  0.0995  0.0485  0.070  0  0  1
2  0.530  0.420  0.135  0.6770  0.2565  0.1415  0.210  1  0  0
3  0.440  0.365  0.125  0.5160  0.2155  0.1140  0.155  0  0  1
4  0.330  0.255  0.080  0.2050  0.0895  0.0395  0.055  0  1  0


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/tales/virtualenvs/ML/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


2\. Regressão logística com $C=10^6$
  + Faça a regressão logística dos dados transformados, com um *C alto* (sem regularização).
  + **Imprima a acurácia** do classificador nos dados de teste com 3 dígitos significativos.
  + Rode o **LogisticRegression** com **random_state=1** para garantir que de o mesmo resultado toda vez que você rodar.

In [4]:
print("\nAcurácia: %.3f" 
    %LogisticRegression(C=10**6, random_state=1).fit(treino, y_treino).score(teste, y_teste))


Acurácia: 0.897


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


3\. Regressão logística com regularização ($C=1$).
  + Imprima com 3 dígitos a acurácia, e use **random_state=1**.

In [5]:
print("\nAcurácia: %.3f" 
    %LogisticRegression(C=1, random_state=1).fit(treino, y_treino).score(teste, y_teste))


Acurácia: 0.892


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


4\. Regressão logística sem regularização e com estandardização dos dados.
  + Use $C=10^6$ mas transforme os dados antes de aplicar a regressão logistica.

In [6]:
print("\nAcurácia: %.3f" 
    %Pipeline([
        ('stdScale', StandardScaler()), 
        ('logistic', LogisticRegression(C=10**6, random_state=1))
    ]).fit(treino, y_treino).score(teste, y_teste))


Acurácia: 0.897


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


5\. Aplique um PCA nos dados, de forma que pelo menos 90% da variância dos dados seja preservada.

In [7]:
# Prepara o PCA que será adicionado no Pipeline, que aplicará as transformações sequencialmente
pca = PCA(n_components=0.9, random_state=1) 

6\. Rode a regressão logística sem regularização nos dados do PCA

In [8]:
print("\nAcurácia: %.3f" 
    %Pipeline([
        ('pca', pca), 
        ('logistic', LogisticRegression(C=10**6, random_state=1))
    ]).fit(treino, y_treino).score(teste, y_teste))


Acurácia: 0.884


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


7\. Rode a regressão logística com regularização ($C=1$) nos dados do PCA

In [9]:
print("\nAcurácia: %.3f" 
    %Pipeline([
        ('pca', pca), 
        ('logistic', LogisticRegression(C=10**6, random_state=1))
    ]).fit(treino, y_treino).score(teste, y_teste))


Acurácia: 0.884


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


8\. Leia o arquivo abalone-missing.csv com dados faltantes na 2ª à penúltima coluna. 
  + Faça o preprocessamento descrito em 1. 
  + Impute pela média os valores faltantes.
  + Rode a regressão sem regularização, sem PCA e sem estandardização.

In [10]:
# Carrega um DataFrame com os dados do arquivo
df = pd.io.parsers.read_csv('abalone-missing.csv', ',', header=None )

# Utiliza a função get_dummies para converter a primeira coluna para um dado numérico
df = pd.get_dummies(df, columns=[0])

# Cria um DataFrame com a versão categórica (1:x>13, 0:x<=13) da coluna 8
maiorQueOito = df.apply(lambda row: int(row[8]>13), 1)
# Remove-a
del df[8]

# Separando dados para treino e teste
treino, y_treino = df[:3133].as_matrix(), maiorQueOito[:3133].as_matrix()
teste, y_teste = df[3133:].as_matrix(), maiorQueOito[3133:].as_matrix()

# Renomeia as colunas para facilitar o acompanhamento
df.columns = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'F', 'I', 'M']

# Mostra as primeiras linhas de nossos dados, para um preview de como estão até então
print("Primeiras 5 linhas:\n", df.head())

# Imputa os valores faltantes pela média e realiza a regressão logística
print("\nAcurácia: %.3f" 
    %Pipeline([
        ('imputer', Imputer()), 
        ('logistic', LogisticRegression(C=10**6, random_state=1))
    ]).fit(treino, y_treino).score(teste, y_teste))

Primeiras 5 linhas:
       V1     V2     V3      V4      V5      V6     V7  F  I  M
0  0.455  0.365  0.095  0.5140  0.2245  0.1010  0.150  0  0  1
1  0.350  0.265  0.090  0.2255  0.0995  0.0485  0.070  0  0  1
2  0.530  0.420  0.135  0.6770  0.2565  0.1415  0.210  1  0  0
3  0.440  0.365  0.125  0.5160  0.2155  0.1140  0.155  0  0  1
4  0.330  0.255  0.080  0.2050  0.0895  0.0395  0.055  0  1  0

Acurácia: 0.887


/home/tales/virtualenvs/ML/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/home/tales/virtualenvs/ML/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/home/tales/virtualenvs/ML/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
